# Waveframe v4.0 — Plug‑and‑Play Quickstart (Colab & local)

**Usage:** *Runtime → Run all*.  
This notebook installs minimal deps, ensures the repo is available, auto-discovers CSVs in `Analysis/` & `Demos/Data/`, and produces plots.  
Figures are saved automatically to `Figures/auto/`.


In [ ]:
# 1) Install + import deps (safe to run multiple times)
!pip -q install numpy pandas matplotlib

import sys, platform, os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print('Python:', sys.version)
print('Platform:', platform.platform())
print('pandas:', pd.__version__)
print('numpy:', np.__version__)
print('matplotlib:', plt.matplotlib.__version__)

In [ ]:
# 2) Figure output setup (auto-creates Figures/ and Figures/auto/)
try:
    plt  # ensure matplotlib is available as plt
except NameError:
    import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

REPO_ROOT = Path(".").resolve()
FIG_DIR = REPO_ROOT / "Figures"
AUTO_DIR = FIG_DIR / "auto"
AUTO_DIR.mkdir(parents=True, exist_ok=True)

def savefig(name: str, fig=None, subdir: str = "auto", dpi=300):
    """Save current (or provided) figure into Figures/<subdir>/ with sane defaults."""
    target_dir = FIG_DIR / subdir if subdir else FIG_DIR
    target_dir.mkdir(parents=True, exist_ok=True)
    fig = fig or plt.gcf()
    out = target_dir / name
    fig.savefig(out, dpi=dpi, bbox_inches="tight")
    print(f"[saved] {out}")
    return out

In [ ]:
# 3) Ensure repo present and set working directory
REPO_URL = 'https://github.com/Wright-Shawn/Waveframe-v4.0.git'
REPO_DIR = '/content/Waveframe-v4.0'

if os.path.isdir('/content') and not os.path.isdir(REPO_DIR):
    print('Cloning repo into Colab workspace...')
    !git clone $REPO_URL

# If running in Colab, switch into the repo
if os.path.isdir(REPO_DIR):
    os.chdir(REPO_DIR)

print('CWD:', os.getcwd())

In [ ]:
# 4) Discover CSVs and load a dataframe
SEARCH_DIRS = ['Analysis', 'analysis', 'Demos/Data']
csvs = []
for rel in SEARCH_DIRS:
    root = os.path.abspath(rel)
    found = sorted(glob.glob(os.path.join(root, '**', '*.csv'), recursive=True))
    if found:
        print(f'Found {len(found)} CSV(s) under {rel}')
        csvs.extend(found)

print('\nAll discovered CSVs:')
for i, p in enumerate(csvs):
    print(f'  [{i}] {p}')

if not csvs:
    print('\nNo CSVs found. Use the upload dialog to add one.')
    try:
        from google.colab import files
        uploaded = files.upload()  # opens a file picker in Colab
        os.makedirs('Uploaded', exist_ok=True)
        for name, data in uploaded.items():
            with open(os.path.join('Uploaded', name), 'wb') as f:
                f.write(data)
            csvs.append(os.path.abspath(os.path.join('Uploaded', name)))
        print('Uploaded CSVs registered:')
        for i, p in enumerate(csvs):
            print(f'  [{i}] {p}')
    except Exception as e:
        raise FileNotFoundError('No CSVs found and upload dialog not available in this environment.')

CSV_INDEX = 0
csv_path = csvs[CSV_INDEX]
print('\nUsing CSV ->', csv_path)
df = pd.read_csv(csv_path)
print('Columns:', list(df.columns))
df.head()

In [ ]:
# 5) Detect columns and generate plots
def pick(candidates, columns):
    for name in candidates:
        if name in columns:
            return name
    return None

Z = pick(['z','redshift','Z'], df.columns)
E = pick(['E','E_z','E(z)'], df.columns)
DA = pick(['D_A','D_A(z)','DA','DA_z','angular_diameter_distance'], df.columns)
H = pick(['H','H_z','H(z)','Hz'], df.columns)
G = pick(['growth','f_sigma8','fσ8','f_s8','fs8'], df.columns)

print('Detected ->', {'z':Z, 'E':E, 'D_A':DA, 'H':H, 'growth':G})

def maybe_plot(xname, yname, title, fname):
    if xname and yname:
        plt.figure()
        plt.plot(df[xname], df[yname])
        plt.xlabel(xname); plt.ylabel(yname); plt.title(title); plt.grid(True)
        savefig(fname)  # <--- use the helper we inserted above
        plt.show()
        return True
    else:
        print(f'Skipping {title}: missing columns')
        return False

maybe_plot(Z, E,   'E(z)',               'ez.png')
maybe_plot(Z, DA,  'D_A(z)',             'da.png')
maybe_plot(Z, H,   'H(z)',               'hz.png')
maybe_plot(Z, G,   'Growth (proxy)',     'growth.png')